In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv()                      # 1️⃣  make env-vars visible

from src.containers import Container   # 2️⃣  class body reads env once

import logging
logging.basicConfig(level=logging.INFO)

container = Container()            # 3️⃣  build the container

weaviate_helper      = container.weaviate_helper()
late_chunking_helper = container.late_chunking_helper()
ingest_helper = container.ingest_helper()


INFO:src.embeddings.late_chunking:Initialized LateChunkingHelper: device=cuda, max_chunk_chars=2048
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jinaai/jina-embeddings-v3
INFO:sentence_transformers.SentenceTransformer:5 prompts are loaded, with the keys: ['retrieval.query', 'retrieval.passage', 'separation', 'classification', 'text-matching']


In [ ]:
ingest_helper._scan_and_get_directories("knowledge\\kubernetes\\website-main\\content\\en\\examples", {"include": ["yaml", "yml"], "exclude": [], "min_length": 0, "max_length": -1, "generate_summary": True})

In [ ]:
with weaviate_helper.connect() as client:
    collection = client.collections.get(name="kubernetes_code")
    for obj in collection.iterator():
        print(obj)

In [ ]:
ingest_helper.ingest_knowledge()

In [ ]:
with weaviate_helper.connect() as client:
    client.collections.delete(name="test")

weaviate_helper.create_collection(collection_name="test", description="test description")

In [ ]:
docs, canonical_doc = late_chunking_helper.chunk_file(
    path="knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md",
)
docs = late_chunking_helper.generate_late_chunking_embeddings(docs, canonical_doc)

In [ ]:
weaviate_helper.batch_insert(docs, collection_name="test")

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Test "HTTP/1.1 200 OK"


In [ ]:
query = "how to design a deployment with horizontal scaling"
query_embedding = late_chunking_helper.generate_query_embedding(query)
rag_results = weaviate_helper.rag_query(collection_name="kubernetes_code", query_embedding=query_embedding, limit=25)
rag_results = [rag_result.properties for rag_result in rag_results]
len(rag_results)


INFO:src.embeddings.late_chunking:generate_query_embedding: query='how to design a deployment with horizontal scaling'
Batches: 100%|██████████| 1/1 [00:00<00:00,  1.36it/s]
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Kubernetes_code "HTTP/1.1 200 OK"


100

In [4]:
ranked_results = late_chunking_helper.re_rank(query=query, docs=rag_results, top_k=10)
ranked_results

INFO:src.embeddings.late_chunking:re_rank: 100 docs, top_k=10
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.29it/s]
INFO:src.embeddings.late_chunking:re_rank: returning 10 ranked docs


[{'doc': {'content': '# This YAML configuration defines a Kubernetes Horizontal Pod Autoscaler (HPA) for the "frontend" application. It automatically adjusts the number of pod replicas based on CPU utilization, aiming to keep it around 50%. The HPA targets a ReplicaSet named "frontend" and maintains a minimum of 3 pods and a maximum of 10 pods. This setup enables dynamic scaling of the application to handle varying workloads efficiently, ensuring optimal performance without over-provisioning.\napiVersion: autoscaling/v1\nkind: HorizontalPodAutoscaler\nmetadata:\n  name: frontend-scaler\nspec:\n  scaleTargetRef:\n    kind: ReplicaSet\n    name: frontend\n  minReplicas: 3\n  maxReplicas: 10\n  targetCPUUtilizationPercentage: 50\n',
   'subchunk': '1/1',
   'summary': 'This YAML configuration defines a Kubernetes Horizontal Pod Autoscaler (HPA) for the "frontend" application. It automatically adjusts the number of pod replicas based on CPU utilization, aiming to keep it around 50%. The HP

In [ ]:
from weaviate.classes.query import Filter

with weaviate_helper.connect() as client:
    collection = client.collections.get(name="test")
    response = collection.query.fetch_objects(
    filters=Filter.by_property("filename").like("*quick-reference.md*"),
    limit=3
)
response.objects

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
d:\Python\MasterIA\TFM\test_folder\.venv\lib\site-packages\weaviate\warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


[Object(uuid=_WeaviateUUIDInt('5dd3f31c-5e73-4bdb-b099-e7fcf1eb1acb'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': '## {{% heading "whatsnext" %}}  \n* Read the [kubectl overview](/docs/reference/kubectl/) and learn about [JsonPath](/docs/reference/kubectl/jsonpath).  \n* See [kubectl](/docs/reference/kubectl/kubectl/) options.  \n* Also read [kubectl Usage Conventions](/docs/reference/kubectl/conventions/) to understand how to use kubectl in reusable scripts.  \n* See more community [kubectl cheatsheets](https://github.com/dennyzhang/cheatsheet-kubernetes-A4).', 'filename': 'knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md', 'chunk': '23/23', 'subchunk': '1/1'}, references=None, vector={}, collection='Test'),
 Object(uuid=_WeaviateUUIDInt('dc2a97aa-b075-49f4-a7ca-1efaeb886585'), metadata=MetadataRetur